In [1]:
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, ActionSelector
from torch.utils.tensorboard import SummaryWriter
from active_rl.utils.demonstration_memory import RolloutOfflineReplayMemory
from active_rl.utils.demonstration_optimization import standard_optimization
from active_rl.networks.dqn_atari import MC_DQN
import torch.optim as optim 
from active_rl.utils.demonstration_training_loop import standard_config, std_demo_config, active_demo_std_training
from active_rl.utils.acquisition_functions import ens_BALD_wrapper

In [2]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [3]:
BATCH_SIZE = 256
LR = 0.0000625
TAU=0.1
NAME='standard_demo_ens_dqn_BALD'

In [4]:
device = 'cuda:0'
policy_net = MC_DQN(n_actions).to(device)
target_net = MC_DQN(n_actions).to(device)
policy_net.apply(policy_net.init_weights)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)

BALD_rank_func = ens_BALD_wrapper(policy_net, TAU, BATCH_SIZE, device)

In [5]:
writer = SummaryWriter(f'runs/{NAME}')

In [6]:
standard_config['file_name'] = './data/Breakout'
standard_config['policy_net'] = policy_net
standard_config['target_net'] = target_net
standard_config['optimizer'] = optimizer
standard_config['device'] = device
standard_config['writer'] = writer
standard_config['batch_size'] = BATCH_SIZE
standard_config['rank_func'] = BALD_rank_func

std_demo_config['env'] = env
std_demo_config['env_raw'] = env_raw
std_demo_config['device'] = device
std_demo_config['policy_net'] = policy_net
std_demo_config['target_net'] = target_net
std_demo_config['optimizer'] = optimizer
std_demo_config['batch_size'] = BATCH_SIZE
std_demo_config['writer'] = writer
std_demo_config['demo_config'] = standard_config

In [ ]:
active_demo_std_training(std_demo_config)